In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import datajoint as dj
import numpy as np


print("neuron tab labeler started new")
#setting the address and the username
print("about to connect to database")
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'stelios'
dj.config['database.password'] = 'stelios#1'
dj.config['display.limit'] = 10
#will state whether words are shown or not
dj.config['safemode']=True
print(dj.conn(reset=True))


neuron tab labeler started new
about to connect to database
Connecting stelios@10.28.0.34:3306
DataJoint connection (connected) stelios@10.28.0.34:3306


In [3]:
pinky = dj.create_virtual_module("pinky","microns_pinky")
pinky_nda = dj.create_virtual_module("pinky_nda","microns_pinky_nda")
schema = dj.schema("microns_pinky")

In [4]:
#pinky.SynapseCompartmentSpineLabelRevised.drop()

In [5]:
@schema
class SynapseCompartmentSpineLabelRevised(dj.Manual):
    definition = """
    -> pinky.Synapse
    segmentation         : smallint                     # segmentation id
    synapse_id           : bigint unsigned              # synapse index within the segmentation
    ---
    spine_label          : tinyint unsigned             # numeric label of the compartment
    segment_id               : bigint unsigned              # segment id unique within each Segmentation
    compartment_label    : tinyint unsigned             # numeric label of the compartment
    """

In [8]:
pinky.SynapseSpineLabelFinalThresholdRevised()

segmentation segmentation id,synapse_id synapse index within the segmentation,spine_label numeric label of the compartment
3,3,13
3,5,13
3,22,13
3,25,13
3,30,13
3,41,13
3,73,13
3,77,13
3,106,13
3,108,13


In [7]:
axon_segments = (pinky.Neurite() & 'neurite_type="axon"').proj(presyn='segment_id') * pinky.CurrentSegmentation()
post_trace_segments = (pinky.Segment() & pinky.CurrentSegmentation() & pinky_nda.Trace()).proj(postsyn='segment_id')

final_synapses_with_comp_labels = ((pinky.Synapse() & post_trace_segments & axon_segments) - pinky.SynapseExclude().proj()) * pinky.SynapseCompartmentLabelFinalThresholdRevised
relevant_synapses_with_comp_labels = final_synapses_with_comp_labels.proj("presyn","postsyn","postsynaptic_label")
complete_synapse_comp_table = ((relevant_synapses_with_comp_labels * pinky.LabelKey().proj('description', postsynaptic_label='numeric')).proj("presyn","postsynaptic_label","description"))

final_synapses_with_spine_labels = ((pinky.Synapse() & post_trace_segments & axon_segments) - pinky.SynapseExclude().proj()) * pinky.SynapseSpineLabelFinalThresholdRevised
relevant_synapses_with_spine_labels = final_synapses_with_spine_labels.proj("presyn","postsyn","spine_label")
complete_synapse_spine_table = ((relevant_synapses_with_spine_labels * pinky.LabelKey().proj('description', spine_label='numeric')).proj("presyn","spine_label","description"))


entries_to_add = dj.U('spine_label')*((dj.U('segmentation', 'synapse_id') & complete_synapse_comp_table) - (dj.U('segmentation', 'synapse_id') & complete_synapse_spine_table)).proj(spine_label='0')
updated_spine_table = complete_synapse_spine_table.proj() + entries_to_add.proj()
complete_spine_compartment_label_table = (relevant_synapses_with_comp_labels.proj('presyn', 'postsynaptic_label') * updated_spine_table).proj('presyn', compartment_label='postsynaptic_label')


final_table = complete_spine_compartment_label_table.proj("compartment_label",segment_id="presyn")
final_table



segmentation segmentation id,synapse_id synapse index within the segmentation,spine_label numeric label of the compartment,segment_id segment id unique within each Segmentation,compartment_label numeric label of the compartment
3,2830887,13,648518346342791653,8
3,2225519,13,648518346342793079,8
3,2974232,13,648518346346298672,8
3,2144008,13,648518346346303291,3
3,2581464,13,648518346342802558,3
3,2678345,13,648518346341399828,3
3,2418200,13,648518346341401226,3
3,3367190,13,648518346348214946,3
3,1875155,13,648518346349369921,8
3,1534460,13,648518346341389996,8


In [10]:
pinky.SynapseCompartmentSpineLabelRevised.insert(final_table)

In [12]:
segment_ids = pinky.SynapseCompartmentSpineLabelRevised.fetch("segment_id")

In [13]:
len(np.unique(segment_ids))

30089